In [8]:
# ! pip install Pillow==6.2.2
! pip show Pillow

Name: Pillow
Version: 7.0.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Alex Clark (PIL Fork Author)
Author-email: aclark@python-pillow.org
License: HPND
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: tflearn, tensor2tensor, magenta, wordcloud, torchvision, scikit-image, lucid, imgaug, imageio, image, fastai, bokeh
Name: Pillow
Version: 7.0.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Alex Clark (PIL Fork Author)
Author-email: aclark@python-pillow.org
License: HPND
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: tflearn, tensor2tensor, magenta, wordcloud, torchvision, scikit-image, lucid, imgaug, imageio, image, fastai, bokeh


In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


# Object Detection

In [2]:
!git clone https://github.com/tensorflow/tpu/

from PIL import Image
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
import sys
sys.path.insert(0, 'tpu/models/official')
sys.path.insert(0, 'tpu/models/official/mask_rcnn')
import coco_metric
from mask_rcnn.object_detection import visualization_utils

min_score_thresh = 0.8 
ID_MAPPING = {
    1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven', 80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock', 86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush',
}
category_index = {k: {'id': k, 'name': ID_MAPPING[k]} for k in ID_MAPPING}

def objectDetetionMain(image_path):
  poster = Image.open(image_path)
  width, height = poster.size
  posterArea = width*height
  imgNp = np.array(poster)
  result = []

  classes, scores, boxes = createSession(image_path)

  for objClass, score, box in zip(classes, scores, boxes): 
    if score > min_score_thresh:
      objName = ID_MAPPING[objClass]
      percentage = f'{(calcObjectArea(box, imgNp)/posterArea*100):.2f}%'
      result.append((objName, percentage))

  return result

Cloning into 'tpu'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 9671 (delta 14), reused 19 (delta 9), pack-reused 9643
Receiving objects: 100% (9671/9671), 24.22 MiB | 26.96 MiB/s, done.
Resolving deltas: 100% (6929/6929), done.
TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
from PIL import Image
import cv2

def calcObjectArea(box, imgNp):
    y, x, b, a = box
    h = b - y
    w = a - x

    cv2.rectangle(imgNp, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return w*h


def createSession(image_path):
  image = Image.open(image_path)
  width, height = image.size
  np_image = np.array(image.getdata()).reshape(height, width, 3).astype(np.uint8)
  session = tf.Session(graph=tf.Graph())

  saved_model_dir = 'gs://cloud-tpu-checkpoints/mask-rcnn/1555659850'
  _ = tf.saved_model.loader.load(session, ['serve'], saved_model_dir)

  return detectObjects(session, image_path, np_image)


def detectObjects(session, image_path, np_image):  
  height, width, d = np_image.shape
  max_boxes_to_draw = 50  

  with open(image_path, 'rb') as f:
    np_image_string = np.array([f.read()])

  num_detections, detection_boxes, detection_classes, detection_scores, detection_masks, image_info = session.run(
      ['NumDetections:0', 'DetectionBoxes:0', 'DetectionClasses:0', 'DetectionScores:0', 'DetectionMasks:0', 'ImageInfo:0'],
      feed_dict={'Placeholder:0': np_image_string})

  num_detections = np.squeeze(num_detections.astype(np.int32), axis=(0,))
  detection_boxes = np.squeeze(detection_boxes * image_info[0, 2], axis=(0,))[0:num_detections]
  detection_scores = np.squeeze(detection_scores, axis=(0,))[0:num_detections]
  detection_classes = np.squeeze(detection_classes.astype(np.int32), axis=(0,))[0:num_detections]
  instance_masks = np.squeeze(detection_masks, axis=(0,))[0:num_detections]
  ymin, xmin, ymax, xmax = np.split(detection_boxes, 4, axis=-1)
  processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)
  segmentations = coco_metric.generate_segmentation_from_masks(instance_masks, processed_boxes, height, width)


  image_with_detections = visualization_utils.visualize_boxes_and_labels_on_image_array(
      np_image,
      detection_boxes,
      detection_classes,
      detection_scores,
      category_index,
      instance_masks=segmentations,
      use_normalized_coordinates=False,
      max_boxes_to_draw=max_boxes_to_draw,
      min_score_thresh=min_score_thresh)

  # img = Image.fromarray(image_with_detections)
  # display(img)
  
  return detection_classes, detection_scores, detection_boxes

# Main

In [4]:
import json
# import PIL
# PIL.Image.MAX_IMAGE_PIXELS = 122760000

def analyzePoster(movieTitle, posterPath):

  print('object detection...')
  objectsDictionary = objectDetetionMain(posterPath)
  # display(objectsDictionary)

  return {
    "objectsDictionary": objectsDictionary,
  }

moviesMetaData = [{'imdbID': '1951264', 'title': 'The Hunger Games: Catching Fire'},{'imdbID': '1300854', 'title': 'Iron Man 3'},{'imdbID': '2294629', 'title': 'Frozen'},{'imdbID': '1690953', 'title': 'Despicable Me 2'},{'imdbID': '0770828', 'title': 'Man of Steel'},{'imdbID': '1454468', 'title': 'Gravity'},{'imdbID': '1453405', 'title': 'Monsters University'},{'imdbID': '1170358', 'title': 'The Hobbit: The Desolation of Smaug'},{'imdbID': '3313776', 'title': 'Fast'},{'imdbID': '1623205', 'title': 'Oz the Great and Powerful'},{'imdbID': '1408101', 'title': 'Star Trek Into Darkness'},{'imdbID': '1981115', 'title': 'Thor: The Dark World'},{'imdbID': '0816711', 'title': 'World War Z'},{'imdbID': '0481499', 'title': 'The Croods'},{'imdbID': '2404463', 'title': 'The Heat'},{'imdbID': '1723121', 'title': "We're the Millers"},{'imdbID': '1800241', 'title': 'American Hustle'},{'imdbID': '1457767', 'title': 'The Conjuring'},{'imdbID': '2024432', 'title': 'Identity Thief'},{'imdbID': '2191701', 'title': 'Grown Ups 2'},{'imdbID': '1430132', 'title': 'The Wolverine'},{'imdbID': '1229340', 'title': 'Anchorman 2: The Legend Continues'},{'imdbID': '1091191', 'title': 'Lone Survivor'},{'imdbID': '1583421', 'title': 'G.I. Joe: Retaliation'},{'imdbID': '1985966', 'title': 'Cloudy with a Chance of Meatballs 2'},{'imdbID': '1670345', 'title': 'Now You See Me'},{'imdbID': '0993846', 'title': 'The Wolf of Wall Street'},{'imdbID': '1327773', 'title': "Lee Daniels' The Butler"},{'imdbID': '1951261', 'title': 'The Hangover Part III'},{'imdbID': '0848537', 'title': 'Epic'},{'imdbID': '1535109', 'title': 'Captain Phillips'},{'imdbID': '1663662', 'title': 'Pacific Rim'},{'imdbID': '1245492', 'title': 'This Is the End'},{'imdbID': '2302755', 'title': 'Olympus Has Fallen'},{'imdbID': '0453562', 'title': '42'},{'imdbID': '1535108', 'title': 'Elysium'},{'imdbID': '1691917', 'title': 'Planes'},{'imdbID': '1210819', 'title': 'The Lone Ranger'},{'imdbID': '1483013', 'title': 'Oblivion'},{'imdbID': '2226417', 'title': 'Insidious: Chapter 2'},{'imdbID': '2140373', 'title': 'Saving Mr. Banks'},{'imdbID': '1860353', 'title': 'Turbo'},{'imdbID': '1272878', 'title': '2 Guns'},{'imdbID': '2334879', 'title': 'White House Down'},{'imdbID': '2023587', 'title': 'Mama'},{'imdbID': '1702439', 'title': 'Safe Haven'},{'imdbID': '2017020', 'title': 'The Smurfs 2'},{'imdbID': '2083355', 'title': 'The Best Man Holiday'},{'imdbID': '1854564', 'title': 'Percy Jackson: Sea of Monsters'},{'imdbID': '1606378', 'title': 'A Good Day to Die Hard'},{'imdbID': '1588173', 'title': 'Warm Bodies'},{'imdbID': '1351685', 'title': 'Jack the Giant Slayer'},{'imdbID': '2184339', 'title': 'The Purge'},{'imdbID': '1204975', 'title': 'Last Vegas'},{'imdbID': '1731141', 'title': "Ender's Game"},{'imdbID': '1392214', 'title': 'Prisoners'},{'imdbID': '1815862', 'title': 'After Earth'},{'imdbID': '0359950', 'title': 'The Secret Life of Walter Mitty'},{'imdbID': '0765446', 'title': 'Escape from Planet Earth'},{'imdbID': '2024544', 'title': '12 Years a Slave'},{'imdbID': '1621039', 'title': 'Free Birds'},{'imdbID': '1821694', 'title': 'RED 2'},{'imdbID': '2609758', 'title': "Tyler Perry's A Madea Christmas"},{'imdbID': '1911644', 'title': 'The Call'},{'imdbID': '1321870', 'title': 'Gangster Squad'},{'imdbID': '3206598', 'title': 'The World of Jurassic Park 3D'},{'imdbID': '2234155', 'title': 'The Internship'},{'imdbID': '2378281', 'title': 'Instructions Not Included'},{'imdbID': '0882977', 'title': 'Snitch'},{'imdbID': '1411250', 'title': 'Riddick'},{'imdbID': '2734660', 'title': 'A Haunted House'},{'imdbID': '1335975', 'title': '47 Ronin'},{'imdbID': '1322269', 'title': 'August: Osage County'},{'imdbID': '2431286', 'title': 'Philomena'},{'imdbID': '1762399', 'title': 'Walking with Dinosaurs 3D'},{'imdbID': '1572315', 'title': 'Texas Chainsaw 3D'},{'imdbID': '0790736', 'title': 'R.I.P.D.'},{'imdbID': '2334873', 'title': 'Blue Jasmine'},{'imdbID': '2609912', 'title': 'Kevin Hart: Let Me Explain'},{'imdbID': '0795461', 'title': 'Scary Movie 5'},{'imdbID': '1538403', 'title': 'The Mortal Instruments: City of Bones'},{'imdbID': '2387559', 'title': 'Delivery Man'},{'imdbID': '1661382', 'title': 'Grudge Match'},{'imdbID': '2515086', 'title': 'One Direction: This Is Us'},{'imdbID': '1650554', 'title': 'Kick-Ass 2'},{'imdbID': '0790636', 'title': 'Dallas Buyers Club'},{'imdbID': '1213663', 'title': "The World's End"},{'imdbID': '1211956', 'title': 'Escape Plan'},{'imdbID': '2229499', 'title': 'Don Jon'},{'imdbID': '0790628', 'title': 'The Incredible Burt Wonderstone'},{'imdbID': '1931435', 'title': 'The Big Wedding'},{'imdbID': '3379694', 'title': 'And the Mud Ship Sails Away...'},{'imdbID': '1171222', 'title': 'Baggage Claim'},{'imdbID': '1727388', 'title': 'The Way Way Back'},{'imdbID': '0816442', 'title': 'The Book Thief'},{'imdbID': '2313774', 'title': 'Homefront'},{'imdbID': '1235522', 'title': 'Broken City'},{'imdbID': '2364841', 'title': 'Runner Runner'},{'imdbID': '3169640', 'title': 'Yozakura Quartet: Hana no Uta'},{'imdbID': '1814621', 'title': 'Admission'},{'imdbID': '1821549', 'title': 'Nebraska'},{'imdbID': '1904996', 'title': 'Parker'},{'imdbID': '2390361', 'title': 'Enough Said'},{'imdbID': '2387433', 'title': 'Dark Skies'},{'imdbID': '2193215', 'title': 'The Counselor'},{'imdbID': '2357129', 'title': 'Jobs'},{'imdbID': '2334649', 'title': 'Fruitvale Station'},{'imdbID': '2194499', 'title': 'About Time'},{'imdbID': '2034139', 'title': 'The Last Exorcism Part II'},{'imdbID': '2042568', 'title': 'Inside Llewyn Davis'},{'imdbID': '1549920', 'title': 'The Last Stand'},{'imdbID': '1206543', 'title': 'Out of the Furnace'},{'imdbID': '2101341', 'title': 'Dead Man Down'},{'imdbID': '4047068', 'title': 'Bullet to the Head: Mayhem Inc.'},{'imdbID': '1532958', 'title': 'Battle of the Year'},{'imdbID': '1333125', 'title': 'Movie 43'},{'imdbID': '2304771', 'title': 'Mandela: Long Walk to Freedom'},{'imdbID': '2209418', 'title': 'Before Midnight'},{'imdbID': '1833673', 'title': 'Dhoom:3'},{'imdbID': '2385006', 'title': 'Jerusalem'},{'imdbID': '2002718', 'title': 'Machete Kills'},{'imdbID': '1413495', 'title': 'Paranoia'},{'imdbID': '3195742', 'title': 'Journey to the South Pacific'},{'imdbID': '1425922', 'title': 'Black Nativity'},{'imdbID': '1714206', 'title': 'The Spectacular Now'},{'imdbID': '1462900', 'title': 'The Grandmaster'},{'imdbID': '2017038', 'title': 'All Is Lost'},{'imdbID': '2009606', 'title': 'Believe'},{'imdbID': '2132285', 'title': 'The Bling Ring'},{'imdbID': '3203890', 'title': 'Pulling Strings'},{'imdbID': '2218003', 'title': 'Closed Circuit'},{'imdbID': '2112124', 'title': 'Chennai Express'},{'imdbID': '2178470', 'title': 'Yeh Jawaani Hai Deewani'},{'imdbID': '1758795', 'title': 'The To Do List'},{'imdbID': '2172935', 'title': 'Metallica Through the Never'},{'imdbID': '1858776', 'title': 'RockBarnes: The Emperor in You'},{'imdbID': '1837703', 'title': 'The Fifth Estate'},{'imdbID': '2294677', 'title': 'In a World...'},{'imdbID': '2051894', 'title': 'Home Run'},{'imdbID': '2358891', 'title': 'The Great Beauty'},{'imdbID': '2215477', 'title': 'Goliyon Ki Rasleela Ram-Leela'},{'imdbID': '2349460', 'title': 'Grace Unplugged'},{'imdbID': '1618432', 'title': "I'm in Love with a Church Girl"},{'imdbID': '1924429', 'title': 'Trance'},{'imdbID': '1869716', 'title': 'The East'},{'imdbID': '2819924', 'title': 'Candice Renoir'},{'imdbID': '2739338', 'title': 'The Christmas Candle'},{'imdbID': '2278871', 'title': 'Blue Is the Warmest Color'},{'imdbID': '1029231', 'title': 'Krrish 3'},{'imdbID': '1985019', 'title': 'Austenland'},{'imdbID': '2887322', 'title': 'The Saratov Approach'},{'imdbID': '2545118', 'title': 'Blackfish'},{'imdbID': '2444946', 'title': 'Girl Rising'},{'imdbID': '3041534', 'title': 'The Attack of the Brain People'},{'imdbID': '1682180', 'title': 'Stoker'},{'imdbID': '2356180', 'title': 'Bhaag Milkha Bhaag'},{'imdbID': '1375789', 'title': 'Race 2'},{'imdbID': '1390398', 'title': 'Bless Me, Ultima'},{'imdbID': '3294904', 'title': 'Disconnect. Reconnect.'},{'imdbID': '2243389', 'title': "I'm So Excited!"},{'imdbID': '2404461', 'title': 'The Past'},{'imdbID': '1778924', 'title': 'The Ultimate Life'},{'imdbID': '2179116', 'title': 'The Kings of Summer'},{'imdbID': '1700845', 'title': 'The Invisible Woman'},{'imdbID': '2215151', 'title': 'Inequality for All'},{'imdbID': '2213054', 'title': 'Kai po che!'},{'imdbID': '1311071', 'title': 'Kill Your Darlings'},{'imdbID': '2370248', 'title': 'Short Term 12'},{'imdbID': '3463312', 'title': 'Ginger'},{'imdbID': '1986769', 'title': 'Alone Yet Not Alone'},{'imdbID': '2205904', 'title': 'Generation Iron'},{'imdbID': '1602613', 'title': 'Only God Forgives'},{'imdbID': '2275802', 'title': 'Satyagraha'},{'imdbID': '2492916', 'title': 'Muscle Shoals'},{'imdbID': '2576450', 'title': 'Besharam'},{'imdbID': '2357377', 'title': 'The Berlin File'},{'imdbID': '2345112', 'title': 'Parkland'},{'imdbID': '2590506', 'title': 'The Hunt'},{'imdbID': '2406422', 'title': 'Walking the Camino: Six Ways to Santiago'},{'imdbID': '1596753', 'title': 'Salinger'},{'imdbID': '2224317', 'title': 'Lootera'},{'imdbID': '1390539','title': 'The Lost Medallion: The Adventures of Billy Stone'},{'imdbID': '1893326', 'title': "Scatter My Ashes at Bergdorf's"},{'imdbID': '3008014', 'title': 'The Face Reader'},{'imdbID': '2084989', 'title': 'Upstream Color'},{'imdbID': '2306745', 'title': 'Sound City'},{'imdbID': '2476154', 'title': 'Grand Masti'},{'imdbID': '2388637', 'title': "Ain't Them Bodies Saints"},{'imdbID': '2532528', 'title': 'Dirty Wars'},{'imdbID': '1638364', 'title': 'The Armstrong Lie'},{'imdbID': '2992152', 'title': 'Personal Tailor'},{'imdbID': '2301155', 'title': 'Shootout at Wadala'},{'imdbID': '2167056', 'title': 'When Comedy Went to School'},{'imdbID': '1426329', 'title': 'Lovelace'},{'imdbID': '2265398', 'title': 'Drinking Buddies'},{'imdbID': '2545338', 'title': 'Citizen Koch'},{'imdbID': '2275794', 'title': 'Gori Tere Pyaar Mein'},{'imdbID': '1758595', 'title': 'Diana'},{'imdbID': '2103267', 'title': 'Adore'},{'imdbID': '1564870', 'title': 'Not Today'},{'imdbID': '2359427', 'title': 'Linsanity'},{'imdbID': '2436516', 'title': 'Go Goa Gone'},{'imdbID': '2571226', 'title': 'The Anonymous People'},{'imdbID': '2344678', 'title': 'Himmatwala'},{'imdbID': '3259324', 'title': 'The Summit'},{'imdbID': '1754811', 'title': 'InAPPropriate Comedy'},{'imdbID': '2437954', 'title': 'Singh Saab the Great'},{'imdbID': '2321163', 'title': 'ABCD (Any Body Can Dance)'},{'imdbID': '2195548', 'title': 'Prince Avalanche'},{'imdbID': '2246724', 'title': 'Ghanchakkar'},{'imdbID': '2332579', 'title': 'Crystal Fairy & the Magical Cactus'},{'imdbID': '2355540', 'title': 'Cutie and the Boxer'},{'imdbID': '1311060', 'title': 'A.C.O.D.'},{'imdbID': '2312890', 'title': 'Afternoon Delight'},{'imdbID': '2553908', 'title': 'Cinco de Mayo, La Batalla'},{'imdbID': '2187884', 'title': 'Escape from Tomorrow'},{'imdbID': '2404555', 'title': 'Copperhead'},{'imdbID': '2369154', 'title': 'R... Rajkumar'},{'imdbID': '1824254', 'title': 'We Steal Secrets: The Story of WikiLeaks'},{'imdbID': '2717558', 'title': 'On the Job'},{'imdbID': '2094890', 'title': 'Mother of George'},{'imdbID': '2622130', 'title': 'Jayantabhai Ki Luv Story'},{'imdbID': '2852400', 'title': 'A Touch of Sin'},{'imdbID': '1718898', 'title': 'Snake and Mongoose'},{'imdbID': '2821880', 'title': 'Rangeelay'},{'imdbID': '2545088', 'title': 'American Promise'},{'imdbID': '2504022', 'title': 'Narco Cultura'},{'imdbID': '2505938', 'title': "Good Ol' Freda"},{'imdbID': '1817287', 'title': 'Is the Man Who Is Tall Happy?'},{'imdbID': '2828840', 'title': 'Fists of Legend'},{'imdbID': '2820820', 'title': 'American Drug War 2: Cannabis Destiny'},{'imdbID': '2051879', 'title': 'Europa Report'},{'imdbID': '2806788', 'title': 'Fukrey'},{'imdbID': '2710368', 'title': 'Final: The Rapture'},{'imdbID': '1785612', 'title': 'The Punk Singer'},{'imdbID': '2554648', 'title': 'Spark: A Burning Man Story'},{'imdbID': '2406676', 'title': 'Ek Thi Daayan'},{'imdbID': '2055765', 'title': 'The English Teacher'},{'imdbID': '1922748', 'title': 'The Investigator'},{'imdbID': '2761118', 'title': 'Stevie Nicks: In Your Dreams'},{'imdbID': '2007360', 'title': 'Computer Chess'},{'imdbID': '3499064', 'title': 'Mamma Therese'},{'imdbID': '3102762', 'title': 'Capital'},{'imdbID': '3332860', 'title': 'Una Noche Inolvidable'},{'imdbID': '2276023', 'title': 'The United States of Autism'},{'imdbID': '2016940', 'title': 'Man of Tai Chi'},{'imdbID': '2027064', 'title': 'Blue Caprice'},{'imdbID': '3573194', 'title': 'Byzantium a Tale of Three Cities'},{'imdbID': '2752724', 'title': 'Money for Nothing: Inside the Federal Reserve'},{'imdbID': '2677816', 'title': 'Settai'},{'imdbID': '2992146', 'title': 'Young Detective Dee: Rise of the Sea Dragon'},{'imdbID': '2901548', 'title': 'The Rooftop'},{'imdbID': '1356790', 'title': 'I Am Divine'},{'imdbID': '2309021', 'title': 'We Are What We Are'},{'imdbID': '2357926', 'title': 'Ramaiya Vastavaiya'},{'imdbID': '3144098', 'title': 'Commitment'},{'imdbID': '2430104', 'title': 'After Tiller'},{'imdbID': '1710469', 'title': 'Seasons of Gray'},{'imdbID': '2247432', 'title': 'Go for Sisters'},{'imdbID': '2555268', 'title': 'Maidentrip'},{'imdbID': '1992193', 'title': "Pandora's Promise"},{'imdbID': '2119463', 'title': 'Let the Fire Burn'},{'imdbID': '2102502', 'title': 'My Lucky Star'},{'imdbID': '1894476', 'title': 'How I Live Now'},{'imdbID': '2489734', 'title': 'The Trials of Muhammad Ali'},{'imdbID': '3989646', 'title': "Faust's Growth"},{'imdbID': '2292959', 'title': 'The Canyons'},{'imdbID': '2084953', 'title': 'Terms and Conditions May Apply'},{'imdbID': '2848824', 'title': 'Bajatey Raho'},{'imdbID': '1650393', 'title': 'C.O.G.'},{'imdbID': '3282770', 'title': 'Leonie Ossowski'},{'imdbID': '2461226', 'title': 'Going Attractions: The Definitive Story of the American Drive-in Movie'},{'imdbID': '1874513', 'title': 'God Loves Uganda'},{'imdbID': '2506388', 'title': 'Mademoiselle C'},{'imdbID': '2461034', 'title': 'GMO OMG'},{'imdbID': '2229842', 'title': 'Table No. 21'},{'imdbID': '2217458', 'title': 'Reaching for the Moon'},{'imdbID': '2710614', 'title': 'American Made Movie'},{'imdbID': '3086582', 'title': 'Tiny Times 2.0'},{'imdbID': '2235248', 'title': 'Andre Gregory: Before and After Dinner'},{'imdbID': '2296697', 'title': 'Concussion'},{'imdbID': '2158531', 'title': 'Holla II'},{'imdbID': '1786751', 'title': 'CBGB'},{'imdbID': '1480295', 'title': 'Killing Season'},{'imdbID': '2466830', 'title': 'The Short Game'},{'imdbID': '2495118', 'title': 'Ip Man: The Final Fight'},{'imdbID': '1462411', 'title': 'Big Sur'},{'imdbID': '1797487', 'title': 'Savannah'},{'imdbID': '2364949', 'title': 'Touchy Feely'},{'imdbID': '2018086', 'title': 'Camille Claudel 1915'},{'imdbID': '1634003', 'title': 'Underdogs'},{'imdbID': '2244901', 'title': 'I Give It a Year'},{'imdbID': '2451742', 'title': 'Running from Crazy'},{'imdbID': '3091552', 'title': 'At Berkeley'},{'imdbID': '1605791', 'title': 'Beyond the Farthest Star'},{'imdbID': '2130282', 'title': 'These Birds Walk'},{'imdbID': '1787067', 'title': 'Fire in the Blood'},{'imdbID': '2689966', 'title': 'Broadway Idiot'},{'imdbID': '0817545', 'title': 'The Power of Few'},{'imdbID': '2393813', 'title': 'Graceland'},{'imdbID': '2821088', 'title': 'Bastards'},{'imdbID': '1709143', 'title': 'The Last Days on Mars'},{'imdbID': '2222206', 'title': 'Dear Mr. Watterson'},{'imdbID': '1941541', 'title': 'Greenwich Village: Music That Defined a Generation'},{'imdbID': '3043630', 'title': 'Tiny Times 1.0'},{'imdbID': '2147728', 'title': 'I Used to Be Darker'},{'imdbID': '3292836', 'title': "M Is for Metamorphose: The ABC's of Death 2"},{'imdbID': '2324012', 'title': 'My Name Is Viola'},{'imdbID': '1986998', 'title': 'Tio Papi'},{'imdbID': '1951216', 'title': 'The Look of Love'},{'imdbID': '2654562', 'title': 'The Ghosts in Our Machine'},{'imdbID': '2282662', 'title': 'Our Nixon'},{'imdbID': '1280725', 'title': 'Wedding Palace'},{'imdbID': '2274570', 'title': 'Bad Milo'},{'imdbID': '2913948', 'title': 'Whensday'},{'imdbID': '2250032', 'title': 'Smash & Grab: The Story of the Pink Panthers'},{'imdbID': '1540741', 'title': 'A Single Shot'},{'imdbID': '1423636', 'title': 'Torn'},{'imdbID': '3116072', 'title': 'Jamel Shabazz Street Photographer'},{'imdbID': '2948266', 'title': "Dealin' with Idiots"},{'imdbID': '2377398', 'title': 'Violet'},{'imdbID': '1807944', 'title': 'As I Lay Dying'},{'imdbID': '2636522', 'title': 'The Unbelievers'},{'imdbID': '2910520', 'title': 'A Wedding Invitation'},{'imdbID': '2289920', 'title': 'Justice Is Mind'},{'imdbID': '1992156', 'title': 'Il Futuro'},{'imdbID': '1798291', 'title': 'This Is Martin Bonner'},{'imdbID': '2419284', 'title': 'Red Obsession'},{'imdbID': '2304426', 'title': 'The Selfish Giant'},{'imdbID': '2193364', 'title': 'I Am Shahid Afridi'},{'imdbID': '1196948', 'title': 'Charlie Countryman'},{'imdbID': '2186663', 'title': '36 Saints'},{'imdbID': '2765340', 'title': 'So Young'},{'imdbID': '1817276', 'title': 'A Resurrection'},{'imdbID': '2261409', 'title': 'Made in Cleveland'},{'imdbID': '1482980', 'title': 'American Meat'},{'imdbID': '2481246', 'title': 'Caucus'},{'imdbID': '2673854', 'title': 'Midget Zombie Takeover'},{'imdbID': '3498042', 'title': 'Elemental Mass'},{'imdbID': '2121304', 'title': 'The Informant'},{'imdbID': '2650642', 'title': 'In the Name Of'},{'imdbID': '2507128', 'title': 'Birth of the Living Dead'},{'imdbID': '1684233', 'title': 'Welcome to the Punch'},{'imdbID': '2294965', 'title': 'Zero Charisma'},{'imdbID': '2272336', 'title': 'Ghost Team One'},{'imdbID': '2339379', 'title': 'Big Words'},{'imdbID': '3139710', 'title': 'White Elephant'},{'imdbID': '2318527', 'title': 'Hell Baby'},{'imdbID': '2386327', 'title': 'The Institute'},{'imdbID': '2049559', 'title': 'The Happy Sad'},{'imdbID': '2033981', 'title': 'Downloaded'},{'imdbID': '2201548', 'title': 'A Teacher'},{'imdbID': '2650054', 'title': 'La Maison de la radio'},{'imdbID': '1741243', 'title': 'Pawn Shop Chronicles'},{'imdbID': '1662293', 'title': 'Nothing Left to Fear'},{'imdbID': '2401007', 'title': 'Desperate Acts of Magic'},{'imdbID': '2275471', 'title': 'A Perfect Man'},{'imdbID': '2169322', 'title': 'Finding Mr. Right'},{'imdbID': '2520398', 'title': 'Assaulted: Civil Rights Under Fire'},{'imdbID': '2371834', 'title': 'Paradise: Hope'},{'imdbID': '2231138', 'title': 'Some Velvet Morning'},{'imdbID': '2224004', 'title': 'Sweetwater'},{'imdbID': '2706264', 'title': 'Rangrezz'},{'imdbID': '3332118', 'title': 'The Playroom'},{'imdbID': '2186781', 'title': 'Five Dances'},{'imdbID': '2884170', 'title': 'Knife Fight'},{'imdbID': '2825924', 'title': 'Revelation Road 2: The Sea of Glass and Fire'},{'imdbID': '2644800', 'title': 'Hôtel Normandy'},{'imdbID': '1462901', 'title': 'All Is Bright'},{'imdbID': '2640474', 'title': 'The House at the End of Time'},{'imdbID': '1817081', 'title': 'A Case of You'},{'imdbID': '1825784', 'title': '3 Geezers!'},{'imdbID': '1551630', 'title': 'Pawn'},{'imdbID': '2281159', 'title': 'Contracted'},{'imdbID': '3504226', 'title': 'Weekend of a Champion'},{'imdbID': '1748199', 'title': 'Silver Circle'},{'imdbID': '2896952', 'title': 'Barefoot in the Kitchen'},{'imdbID': '3335074', 'title': 'Tricked'},{'imdbID': '2012011', 'title': 'Scenic Route'},{'imdbID': '1684548', 'title': 'Axe Giant: The Wrath of Paul Bunyan'},{'imdbID': '1906329', 'title': 'As Cool as I Am'},{'imdbID': '2226519', 'title': 'Plush'},{'imdbID': '1486843', 'title': 'Drew: The Man Behind the Poster'},{'imdbID': '1735279', 'title': 'A Girl and a Gun'},{'imdbID': '0837559', 'title': 'Approaching Midnight'},{'imdbID': '2709606', 'title': 'Last Flight of the Champion'},{'imdbID': '2070852', 'title': 'The Happy House'},{'imdbID': '1705111', 'title': 'InSpectres'},{'imdbID': '2734552', 'title': "You Don't Need Feet to Dance"},{'imdbID': '1925479', 'title': 'Officer Down'},{'imdbID': '3216182', 'title': 'Bharat Ka Veer Putra - Maharana Pratap'},{'imdbID': '2083231', 'title': 'Java Heat'},{'imdbID': '2402163', 'title': 'Paris Countdown'},{'imdbID': '2736254', 'title': 'Absence'},{'imdbID': '1487931', 'title': 'Khumba'},{'imdbID': '1800367', 'title': 'Remnants'},{'imdbID': '2537176', 'title': 'I Spit on Your Grave 2'},{'imdbID': '0762138', 'title': 'Syrup'},{'imdbID': '2193418', 'title': 'Hammer of the Gods'},{'imdbID': '2278668', 'title': 'Triumph of the Wall'}]

dir = '/content/gdrive/My Drive/עבודות/תואר שני/עבודת גמר/data/movies/imdb'
filename = '/content/gdrive/My Drive/עבודות/תואר שני/עבודת גמר/results/result-object.json'
result = {}
# try:
#   with open(filename) as f:
#     result = json.load(f)
# except:
#   result = {}

for movie in [{'imdbID': '1951264', 'title': 'The Hunger Games: Catching Fire'}]:#moviesMetaData:
  posterName = movie["title"]
  
  try:
    print(f'now analyzing {posterName}')
    result[posterName] = analyzePoster(posterName, f'{dir}/{posterName}.jpg')
  except:
    print("#####################################################")
    print(f'{posterName} got an error')
    print("#####################################################")
    continue
  
  # with open(filename, 'w') as outfile:
  #     json.dump(result, outfile)
  # print('saved json !')

result

now analyzing The Hunger Games: Catching Fire
object detection...
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


{'The Hunger Games: Catching Fire': {'objectsDictionary': [('person',
    '60.28%')]}}